# Mount google drive

In [1]:
!nvidia-smi

Tue Aug 22 17:49:07 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.01              Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080        On  | 00000000:01:00.0 Off |                  N/A |
|  0%   38C    P8               9W / 198W |      0MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Import dependencies

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip install jsons
!pip install langchain
!pip install textwrap3

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number

In [4]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInstructEmbeddings

from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from langchain.document_loaders import PyPDFLoader
from transformers import pipeline
import json
import textwrap

/home/voyagerke/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Ingest pdf file

In [53]:
pdf_file_path = "./productdesc-supportplans.pdf"
# pdf_file_path = "./attention is all you need.pdf"

In [54]:
pdf_loader = PyPDFLoader(pdf_file_path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
splitted_docs = text_splitter.split_documents(pdf_loader.load())

In [55]:
print(splitted_docs)

[Document(page_content='支持计划\n产品介绍文档版本 01 发布日期 2022-11-23\n华为技术有限公司', metadata={'source': './productdesc-supportplans.pdf', 'page': 0}), Document(page_content='版权所有  © 华为技术有限公司  2023。 保留一切权利。\n非经本公司书面许可，任何单位和个人不得擅自摘抄、复制本文档内容的部分或全部，并不得以任何形式传\n播。 商标声明\n和其他华为商标均为华为技术有限公司的商标。\n本文档提及的其他所有商标或注册商标，由各自的所有人拥有。 注意\n您购买的产品、服务或特性等应受华为公司商业合同和条款的约束，本文档中描述的全部或部分产品、服务或\n特性可能不在您的购买或使用范围之内。除非合同另有约定，华为公司对本文档内容不做任何明示或暗示的声\n明或保证。\n由于产品版本升级或其他原因，本文档内容会不定期进行更新。除非另有约定，本文档仅作为使用指导，本文', metadata={'source': './productdesc-supportplans.pdf', 'page': 1}), Document(page_content='档中的所有陈述、信息和建议不构成任何明示或暗示的担保。       华为技术有限公司地址：深圳市龙岗区坂田华为总部办公楼  邮编： 518129 网址：https://www.huawei.com客户服务邮箱：support@huawei.com客户服务电话：4008302118文档版本  01 (2022-11-23)版权所有  © 华为技术有限公司i', metadata={'source': './productdesc-supportplans.pdf', 'page': 1}), Document(page_content='目 录\n1 什么是支持计划 .......................................................................................................................... 1\n2 产品规格 ..........................

# Build and store embeddings

In [10]:
!pip install InstructorEmbedding
!pip install sentence_transformers
!pip install Chromadb

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number

In [11]:
EMBEDDING_MODEL_NAME = "/mnt/h/instructor-large"
embeddings = HuggingFaceInstructEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={"device": "cuda"},
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [37]:
collection_name = 'llama2_demo'
db = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
    persist_directory='./'
)


['f126ebd6-40e5-11ee-b637-19b1a6324d3f',
 'f126ebd7-40e5-11ee-b637-19b1a6324d3f',
 'f126ebd8-40e5-11ee-b637-19b1a6324d3f',
 'f126ebd9-40e5-11ee-b637-19b1a6324d3f',
 'f126ebda-40e5-11ee-b637-19b1a6324d3f',
 'f126ebdb-40e5-11ee-b637-19b1a6324d3f',
 'f126ebdc-40e5-11ee-b637-19b1a6324d3f',
 'f126ebdd-40e5-11ee-b637-19b1a6324d3f',
 'f126ebde-40e5-11ee-b637-19b1a6324d3f',
 'f126ebdf-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe0-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe1-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe2-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe3-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe4-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe5-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe6-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe7-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe8-40e5-11ee-b637-19b1a6324d3f',
 'f126ebe9-40e5-11ee-b637-19b1a6324d3f',
 'f126ebea-40e5-11ee-b637-19b1a6324d3f',
 'f126ebeb-40e5-11ee-b637-19b1a6324d3f',
 'f126ebec-40e5-11ee-b637-19b1a6324d3f',
 'f126ebed-40e5-11ee-b637-19b1a6324d3f',
 'f126ebee-40e5-

In [56]:
db.add_documents(splitted_docs)

['55d03aac-40ea-11ee-b637-19b1a6324d3f',
 '55d03aad-40ea-11ee-b637-19b1a6324d3f',
 '55d03aae-40ea-11ee-b637-19b1a6324d3f',
 '55d03aaf-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab0-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab1-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab2-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab3-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab4-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab5-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab6-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab7-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab8-40ea-11ee-b637-19b1a6324d3f',
 '55d03ab9-40ea-11ee-b637-19b1a6324d3f',
 '55d03aba-40ea-11ee-b637-19b1a6324d3f',
 '55d03abb-40ea-11ee-b637-19b1a6324d3f',
 '55d03abc-40ea-11ee-b637-19b1a6324d3f',
 '55d03abd-40ea-11ee-b637-19b1a6324d3f',
 '55d03abe-40ea-11ee-b637-19b1a6324d3f',
 '55d03abf-40ea-11ee-b637-19b1a6324d3f',
 '55d03ac0-40ea-11ee-b637-19b1a6324d3f',
 '55d03ac1-40ea-11ee-b637-19b1a6324d3f',
 '55d03ac2-40ea-11ee-b637-19b1a6324d3f',
 '55d03ac3-40ea-11ee-b637-19b1a6324d3f',
 '55d03ac4-40ea-

# Test search in Chroma

In [59]:
test_query = '华为云的计费方式是什么样的？'
# test_query = 'what is dominant sequence transduction models?'
search_docs = db.similarity_search(test_query)
print(search_docs)

[Document(page_content='透過紅利分享保誠分紅保單業務基金的利潤分紅計劃小冊子', metadata={'page': 0, 'source': './with-profits-plan-booklet.pdf'}), Document(page_content='\u0bc5ʱϗू\nٙ\n\u0bc5ʱϗू', metadata={'page': 4, 'source': './with-profits-plan-booklet.pdf'}), Document(page_content='●云服务部署实践：侧重云服务部署，包括云服务资源规格选型、云服务使用方式\n等。\n●云服务使用限制：列举云服务使用限制，包括云服务资源配额、带宽限速等。支持计划级别包年购买可获服务配额企业级赠送1次\n 说明\n包年购买企业级支持计划方可享受该服务，需提前 3个工作日提出申请。注意\n可用性检查仅包年购买企业级支持计划才限额赠送。客户一旦申请该限额增值服务，\n将无法退订或降级当期支持计划订单。\n5.6 资源监控与优化\n针对云上资源的告警情况、负载情况和健康情况进行检查。从监控的角度分析业务场\n景和历史故障，结合华为云运维最佳实践，提供针对性的优化建议。\n资源监控与优化包括：', metadata={'page': 13, 'source': './productdesc-supportplans.pdf'}), Document(page_content='目 录\n1 什么是支持计划 .......................................................................................................................... 1\n2 产品规格 ......................................................................................................................................2', metadata={'page': 2, 'source': './productdesc-supportplans.pdf'

In [33]:
# db.delete_collection()

# Build model and transformer pipeline

In [14]:
!pip install accelerate
!pip install bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Exp

In [15]:
model_path = '/mnt/h/Chinese-Llama-2-7b-4bit'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map='auto'
)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:31<00:00, 45.89s/it]


In [16]:
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     torch_dtype=torch.bfloat16,
#     device_map='auto',
#     max_new_tokens=512,
#     do_sample=True,
#     top_k=30,
#     num_return_sequences=1,
#     tokenizer=tokenizer,
#     eos_token_id = tokenizer.eos_token_id
# )
generation_config = GenerationConfig.from_pretrained(model_path)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    generation_config=generation_config,
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [17]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [18]:
instruction = "What is the temperature in Melbourne?"

get_prompt(instruction)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nWhat is the temperature in Melbourne?[/INST]"

In [19]:
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})
system_prompt = "You are an advanced assistant that excels at translation. "
instruction = "Convert the following text from English to French:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an advanced assistant that excels at translation. 
<</SYS>>

Convert the following text from English to French:

 {text}[/INST]


In [20]:
text = "How are you today?"
output = llm_chain.run(text)

/home/voyagerke/.local/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [21]:
def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

In [22]:
parse_text(output)

 Quel est votre état aujourd'hui?




In [23]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [24]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer,\
just say that you don't know, don't try to make up an answer.

{context}

{history}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
memory = ConversationBufferMemory(input_key='question', memory_key='history')

In [41]:
import logging
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [43]:
from langchain.retrievers.multi_query import MultiQueryRetriever

# retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm=llm)
retriever_from_llm = db.as_retriever()

In [52]:
retri_docs = retriever_from_llm.get_relevant_documents('什么是遗留资产?')
print(retri_docs)

[Document(page_content='透過紅利分享保誠分紅保單業務基金的利潤分紅計劃小冊子', metadata={'page': 0, 'source': './with-profits-plan-booklet.pdf'}), Document(page_content='\u0bc5ʱϗू\nٙ\n\u0bc5ʱϗू', metadata={'page': 4, 'source': './with-profits-plan-booklet.pdf'}), Document(page_content='在適當情況下 ， 股票類別證券的投資比例可能會因應市場情況轉變而調節 。例如當利率偏低 ， 有關股票投資比例亦將較低 ，  \n而在利率上升時比例則會較高 （受限於長期目標股票資產分配） 。ׂ\n0%ˢԷ\nᗇᗇ\n在一般情況下 ，保證回報愈低 ， 投資在股票的比例愈高。', metadata={'page': 7, 'source': './with-profits-plan-booklet.pdf'}), Document(page_content='委任精算師作出書面聲明 ， 確保已採取適當措施有效管理我們對保單持有人及股東的責任之間的潛在利益衝突 ， 尤其於派發\n給保單持有人紅利的相關部分。\n此外 ， 我們亦有指定委員會審視投資策略和表現 ， 並監察相關基金的實際表現 。k', metadata={'page': 6, 'source': './with-profits-plan-booklet.pdf'})]


In [45]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    retriever = retriever_from_llm,
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt, "memory": memory}
)

In [47]:
query = input("\nEnter a query: ")
res = qa(query)
print(res)


Enter a query:  What's the model architecture of attention?


{'query': "What's the model architecture of attention?", 'result': ' The attention mechanism used in the paper is a self-attention layer followed by a point-wise feed-forward network. This combines the advantages of both approaches while reducing the overall computational complexity compared to traditional multi-head attention models. ', 'source_documents': [Document(page_content='Attention mechanisms have become an integral part of compelling sequence modeling and transduc-\ntion models in various tasks, allowing modeling of dependencies without regard to their distance in\nthe input or output sequences [ 2,19]. In all but a few cases [ 27], however, such attention mechanisms', metadata={'page': 1, 'source': './attention is all you need.pdf'}), Document(page_content='In this work we employ h= 8 parallel attention layers, or heads. For each of these we use\ndk=dv=dmodel/h= 64 . Due to the reduced dimension of each head, the total computational cost\nis similar to that of single-head at

In [ ]:
while True:
    query = input("\nEnter a query: ")
    if query == "exit":
        break
    res = qa(query)
    answer, docs = res[""]

In [ ]:
def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs